In [1]:
import torch
import numpy
from torchvision.datasets import MNIST,CIFAR100
from torch import nn
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:
transform2 = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

In [3]:
cifar_train = CIFAR100(root = './',train=True,download=True,transform=transform2)

Files already downloaded and verified


In [4]:
cifar_test = CIFAR100(root='./',train=False,transform=transform2)

In [4]:
mnist_train = MNIST(root = './MNIST/train/',train=True,download=True)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1131)>



100%|█████████████████████████████| 9912422/9912422 [00:19<00:00, 510312.15it/s]


Extracting ./MNIST/train/MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/train/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1131)>



100%|██████████████████████████████████| 28881/28881 [00:00<00:00, 93947.12it/s]


Extracting ./MNIST/train/MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/train/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1131)>



100%|█████████████████████████████| 1648877/1648877 [00:03<00:00, 488228.46it/s]


Extracting ./MNIST/train/MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/train/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1131)>



100%|██████████████████████████████████| 4542/4542 [00:00<00:00, 1355330.73it/s]

Extracting ./MNIST/train/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/train/MNIST/raw



In [29]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

In [30]:
mnist_train = MNIST(root = './MNIST/train/',train=True,transform=transform)
mnist_test = MNIST(root = './MNIST/train/',train=False,transform=transform)

In [31]:
train_loader = DataLoader(mnist_train,batch_size = 10,shuffle=True)
test_loader = DataLoader(mnist_test,batch_size = 1,shuffle=True)


In [5]:
train_loader = DataLoader(cifar_train,batch_size = 100,shuffle=True)
test_loader = DataLoader(cifar_test,batch_size= 1, shuffle=True)

### Questions

- [ ] Batch norm vs layer norm
- [ ] How number of kernels affect the models performance
- [ ] How much non-linearity is required for the model?
- [ ] Why restrict our model to just one non-linear function? What happens when multiple non linear functions are used?
- [ ] What if, in CNN layer, activation is applied before pooling and after pooling?


In [ ]:
#relu, sigmoid,tanh, hard tanh, Leaky relu, gelu, 

In [6]:
##Batch norm vs layer norm
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=5,kernel_size=3,stride=1,padding=0,dilation=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(in_channels=5,out_channels=5,kernel_size=5,stride=1,padding=0,dilation=1)
        self.fc1 = nn.Linear(5*11*11,200)
        self.fc2 = nn.Linear(200,400)
        self.fc3 = nn.Linear(400,100)
        self.act1 = F.relu
        self.act2 = F.relu
        self.act3 = F.relu
        self.act4 = F.relu
    def forward(self,x):
        x = self.pool(self.act1(self.conv1(x)))
        x = self.act2(self.conv2(x))
        x = torch.flatten(x,1)
        x = self.act3(self.fc1(x))
        x = self.act4(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x
        
        
        

In [7]:
model = Model()

In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [11]:
for epoch in range(5): 
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training')

/tmp/ipykernel_50469/2365960597.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc3(x))


[1,   200] loss: 4.605
[1,   400] loss: 4.605
[2,   200] loss: 4.605
[2,   400] loss: 4.605
[3,   200] loss: 4.605
[3,   400] loss: 4.605
[4,   200] loss: 4.605


KeyboardInterrupt: 

In [45]:
correct = 0
total = 0
model.eval()
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

/tmp/ipykernel_4525/2365960597.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc3(x))


Accuracy of the network on the 10000 test images: 0 %


In [ ]:
def hook_fn(module, input, output):
    global layer_output
    layer_output = output
handle = model.conv1.register_forward_hook(hook_fn)